In [7]:
import torch
import numpy as np
import sympy as sp
import os, sys
import symbolicregression
import requests
from IPython.display import display

In [8]:
model_path = "model.pt" 
try:
    if not os.path.isfile(model_path): 
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    

cuda:0
Model successfully loaded!


In [14]:
save_path = "/home/takeru/AlphaSymbol/models/meta/model.pkl"
import pickle

with open(save_path, 'wb') as f:
    pickle.dump(model, f)

collections.OrderedDict

In [4]:
est = symbolicregression.model.SymbolicTransformerRegressor(
                        model=model,
                        max_input_points=200,
                        n_trees_to_refine=100,
                        rescale=True
                        )

In [7]:
##Example of data
# Pred Expr:  C(R(R(R(P(1, 1), P(2, 1), Z()), R(S(), P(3, 2), S()), Z()), C(C(S(), S()), C(S(), P(2, 1))), Z()), C(R(P(1, 1), R(P(1, 1), P(3, 1), S()), Z()), R(R(P(1, 1), P(2, 1), Z()), P(3, 2), R(P(1, 1), P(2, 1), Z()))))
#Correct Expr:  C(R(R(R(P(1, 1), P(2, 1), Z()), R(S(), P(3, 2), S()), Z()), C(C(S(), S()), C(S(), P(2, 1))), Z()), C(R(P(1, 1), P(2, 1), Z()), P(2, 2)))
# test values [[78, 44], [7, 40], [24, 8], [30, 1], [18, 5], [28, 41], [14, 23], [42, 32], [6, 51], [1, 38]]
# pred_y_values [83, 75, 11, 0, 5, 77, 41, 59, 97, 71]
# corredt_y_values [89, 81, 17, 0, 11, 83, 47, 65, 103, 77]

x = np.array([[1, 0], [10, 7], [0, 0], [5, 7], [1, 1], [4, 1], [2, 0], [1, 0], [0, 0], [3, 4], [14, 13], [2, 2], [8, 4], [4, 11]])
y = np.array([0, 15, 0, 15, 0, 0, 0, 0, 0, 9, 27, 5, 9, 23])


In [8]:
est.fit(x,y)
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
display(sp.parse_expr(model_str))

(5.1379851551912223*x_1 + 6.850053017174206)*(0.21100000000000002*arctan(6.2077061086153906*x_0 + 20.787431607044485*x_1 - 7.8092440232907647) + 0.00567)

# Use function

In [34]:
x_0, x_1 = sp.symbols("x_0 x_1")
exp_str = "((0.00567 + (0.21100000000000002 * arctan(((-0.299 + (6.66 * (-0.9889463444961373 + (0.25173179678083496 * x_0)))) + ((((26.900000000000002 * (-0.9889463444961373 + (0.25173179678083496 * x_0))) - (2.5300000000000002 + (8.9 * (-0.9889463444961373 + (0.25173179678083496 * x_0))))) + (92.8 + (-5.11 * (-0.8495345825382311 + (0.23786968311070472 * x_1))))) - (-0.8480000000000001 + (-92.5 * (-0.8495345825382311 + (0.23786968311070472 * x_1))))))))) * (25.2 + (21.6 * (-0.8495345825382311 + (0.23786968311070472 * x_1)))))"
expr = sp.simplify(exp_str)
x_values = [[19, 92], [20, 26], [33, 42], [12, 2], [7, 1], [11, 62], [19, 46], [5, 31], [19, 2], [8, 41]]
# pred_y_values [83, 75, 11, 0, 5, 77, 41, 59, 97, 71]
# corredt_y_values [89, 81, 17, 0, 11, 83, 47, 65, 103, 77]
exprs = []
for x, y in x_values:
    exprs.append(expr.subs({x_0: x, x_1: y}))
cur_exprs = [expr.subs(sp.Function("arctan"), sp.atan) for expr in exprs]
print([cur_expr.evalf() for cur_expr in cur_exprs])

[161.608335278932, 47.2975496051804, 75.0116609898571, 5.73994144166445, 3.99644523122401, 109.645795440516, 81.9356637827079, 55.9504419924757, 5.74950149498137, 73.2721772309043]
